In [3]:
%reload_ext autoreload
%autoreload 2
%cd llama3/llama

[Errno 2] No such file or directory: 'llama3/llama'
/home/tbai4/llama3/llama


/home/tbai4/llama3/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [34]:
import os
import torch
from llama import Workflow, Llama

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

workflow = Workflow.build(
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=512*16,
    max_batch_size=4,
    model_parallel_size=1,
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    max_nodes=100,
    max_parents=10,
)

llama = Llama(workflow.model, workflow.tokenizer)

KeyboardInterrupt: 

## Sanity check KV equality

In [17]:
import torch

completion = llama.chat_completion([[
    {'role': 'system', 'content': 'Answer the user\'s question please.'},
    {'role': 'user', 'content': 'What is the capital of France?'},
    {'role': 'user', 'content': 'What is the capital of Germany?'},
]])

print(completion)

regular_cache_v = llama.model.layers[0].attention.cache_v[0].clone()
regular_cache_k = llama.model.layers[0].attention.cache_k[0].clone()

workflow.reset()

[system] = workflow.insert([
    {
        'message': {'role': 'system', 'content': 'Answer the user\'s question please.'},
        'parent_ids': [],
    },
])

[user_1] = workflow.insert([
    {
        'message': {'role': 'user', 'content': 'What is the capital of France?'},
        'parent_ids': [system['id']],
    },
])

[user_2] = workflow.insert([
   {
        'message': {'role': 'user', 'content': 'What is the capital of Germany?'},
        'parent_ids': [system['id'], user_1['id']],
    }
])

workflow_cache_v = workflow.model.layers[0].attention.cache_v[0].clone()
workflow_cache_k = workflow.model.layers[0].attention.cache_k[0].clone()

for n in range(workflow.cache_len):
    print(n, (workflow_cache_v[n] - regular_cache_v[n]).max())
    print(n, (workflow_cache_k[n] - regular_cache_k[n]).max())
    
workflow.model = workflow.model.bfloat16()
llama.model = llama.model.bfloat16()

[{'generation': {'role': 'assistant', 'content': "It seems like you've asked two separate questions. I'll answer both:\n\n1. The capital of France is Paris.\n2. The capital of Germany is Berlin."}}]
0 tensor(0., device='cuda:0')
0 tensor(0., device='cuda:0')
1 tensor(0.0002, device='cuda:0')
1 tensor(0.0020, device='cuda:0')
2 tensor(0.0020, device='cuda:0')
2 tensor(0.0312, device='cuda:0')
3 tensor(6.1035e-05, device='cuda:0')
3 tensor(0.0010, device='cuda:0')
4 tensor(0.0005, device='cuda:0')
4 tensor(0.0312, device='cuda:0')
5 tensor(0.0010, device='cuda:0')
5 tensor(0.0312, device='cuda:0')
6 tensor(0.0005, device='cuda:0')
6 tensor(0.0156, device='cuda:0')
7 tensor(0.0010, device='cuda:0')
7 tensor(0.0312, device='cuda:0')
8 tensor(0.0010, device='cuda:0')
8 tensor(0.0156, device='cuda:0')
9 tensor(0.0005, device='cuda:0')
9 tensor(0.0156, device='cuda:0')
10 tensor(0.0010, device='cuda:0')
10 tensor(0.0156, device='cuda:0')
11 tensor(0.0005, device='cuda:0')
11 tensor(0.0312, de

## Case 1: Parallel encoding, no compaction

In [5]:
workflow.reset()

[system_1] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the user\'s questions.'''},
        'parent_ids': [],
    }
])

q1, q2 = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the capital of Germany?'},
        'parent_ids': [system_1['id']],
    },
    {
        'message': {'role': 'assistant', 'content': 'What is the smallest city in France?'},
        'parent_ids': [system_1['id']],
    },
])

tokens, cached = workflow.step(
    tasks=(
        list({'parent_ids': [system_1['id'], q1['id'], q2['id']], 'expects': ('assistant', None)}
        for _ in range(16))
    ),
    compact=False,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

example #1
The smallest city in France is Saint-Pierre-de-Chartreuse, which has a population of less than 1,000 people. However, another contender for the smallest city in France is Saint-Georges-d'Aurac, which has a population of around 700 people.

If you're looking for a larger city, the smallest city in France by population is likely to be a matter of debate, as there are several small cities with populations under 10,000. Some examples include:

* Saint-Pierre-de-Chartreuse (less than 1,000)
* Saint-Georges-d'Aurac (around

example #2
The smallest city in France is Saint-Benoît, located on the island of Réunion in the Indian Ocean. However, if you're referring to metropolitan France, the smallest city is probably Saint-Nazaire-Blanchefort, which has a population of less than 100 people.

However, if you're looking for a city with a more traditional definition (i.e., a city with a mayor and a defined city limits), then the smallest city in metropolitan France would be Saint-Cirq-La

# Case 2: Parallel encoding, compaction Q1 -> Q2

In [11]:
workflow.reset()

[system] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the questions.'''},
        'parent_ids': [],
    }
])

q1, q2 = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the capital of Germany?'},
        'parent_ids': [system['id']],
    },
    {
        'message': {'role': 'assistant', 'content': 'What is the smallest city in France?'},
        'parent_ids': [system['id']],
    },
])

tokens, cached = workflow.step(
    tasks=[
        {'parent_ids': [system['id'], q1['id']], 'expects': ('assistant', None)}
        for _ in range(16)
    ],
    compact=True,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

example #1
The capital of Germany is Berlin.

example #2
Berlin is the capital of Germany.

example #3
The capital of Germany is Berlin.

example #4
The capital of Germany is Berlin.

example #5
The capital of Germany is Berlin.

example #6
The capital of Germany is Berlin.

example #7
The capital of Germany is Berlin.

example #8
The capital of Germany is Berlin.

example #9
The capital of Germany is Berlin.

example #10
The capital of Germany is Berlin.

example #11
The capital of Germany is Berlin.

example #12
The capital of Germany is Berlin.

example #13
The capital of Germany is Berlin.

example #14
The capital of Germany is Berlin.

example #15
The capital of Germany is Berlin.

example #16
The capital of Germany is Berlin.



# Case 3: Parallel encoding, compaction Q2 -> Q1

In [27]:
workflow.reset()

[system] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the questions.'''},
        'requirements': [],
    }
])

q1, q2 = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the capital of Germany?'},
        'requirements': [system],
    },
    {
        'message': {'role': 'assistant', 'content': 'What is the smallest city in France?'},
        'requirements': [system],
    },
])

tokens, ids, _ = workflow.step(
    tasks=[
        {'requirements': [system, q2, q1], 'expects': ('assistant', None)}
        for _ in range(16)
    ],
    prefill=True,
    compact=True,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

Not fully implemented, compact with precaution!
example #1
The capital of Germany is Berlin.

example #2
Berlin is the capital of Germany.

example #3
The capital of Germany is Berlin.

example #4
The capital of Germany is Berlin.

example #5
The capital of Germany is Berlin.

example #6
The capital of Germany is Berlin. 

What's your next question?

example #7
The capital of Germany is Berlin.

example #8
The capital of Germany is Berlin.

example #9
The capital of Germany is Berlin.

example #10
The capital of Germany is Berlin.

example #11
The capital of Germany is Berlin.

example #12
The capital of Germany is Berlin.

example #13
The capital of Germany is Berlin.

example #14
The capital of Germany is Berlin.

example #15
The capital of Germany is Berlin.

example #16
The capital of Germany is Berlin.



## Case 4: Causal encoding, Q1 -> Q2

In [29]:
workflow.reset()

[system] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the questions.'''},
        'requirements': [],
    }
])

[q1] = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the capital of Germany?'},
        'requirements': [system],
    },
])

[q2] = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the smallest city in France?'},
        'requirements': [system, q1],
    }
])

tokens, ids, _ = workflow.step(
    tasks=[
        {'requirements': [system, q1, q2], 'expects': ('assistant', None)}
        for _ in range(16)
    ],
    prefill=True,
    compact=False,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

example #1
I can answer multiple questions. What would you like to know?

example #2
The capital of Germany is Berlin.

The smallest city in France is Thyez, which has a population of 641 people.

example #3
The capital of Germany is Berlin.

The smallest city in France is Saint-Pierre, which is located on the island of Saint-Pierre in the South Atlantic Ocean. It has a population of around 5,000 and is an overseas collectivity of France.

example #4
The capital of Germany is Berlin.

I'm not aware of any information about the smallest city in France. If you could provide more context or clarify the question, I'll do my best to assist you.

What's your next question?

example #5
The capital of Germany is Berlin.

I don't have enough information to answer the question about the smallest city in France.

example #6
I'm happy to answer a question. However, I don't have any information about a specific question. If you provide a question, I'll do my best to provide a helpful answer.

If yo

## Case 5: Causal encoding, mask out Q1

In [31]:
workflow.reset()

[system] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the questions.'''},
        'requirements': [],
    }
])

[q1] = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the capital of Germany?'},
        'requirements': [system],
    },
])

[q2] = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the smallest city in France?'},
        'requirements': [system, q1],
    }
])

tokens, ids, _ = workflow.step(
    tasks=[
        {'requirements': [system, q2], 'expects': ('assistant', None)}
        for _ in range(16)
    ],
    prefill=True,
    compact=False,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

example #1
The capital of France is the city of The city is Paris, but the smallest city in France is Béthune.

example #2
I can answer your questions.

The smallest city in France is Sarliève.

example #3
I'll answer your questions.

The smallest city in France is Tallinn

example #4
I can answer a question directly. The capital of France is Paris.

example #5
I can answer all of your questions.

example #6
The capital of France is a small town with a population of 4.

example #7
The capital and largest city in Europe

example #8
The capital of France? 

The answer is a required field. 

The answer is Bayonne.

example #9
Berlin is the Sister states

example #10
The capital of the world is Lezat.

example #11
I can answer all of the questions you have, but you didn't provide any questions. Please go ahead and ask your questions, and I'll do my best to provide helpful and accurate answers.

example #12
The capital and largest city in the world?

example #13
I will answer all your quest

## Case 6: Causal encoding, mask Q2

In [32]:
workflow.reset()

[system] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the questions.'''},
        'requirements': [],
    }
])

[q1] = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the capital of Germany?'},
        'requirements': [system],
    },
])

[q2] = workflow.insert([
    {
        'message': {'role': 'assistant', 'content': 'What is the smallest city in France?'},
        'requirements': [system, q1],
    }
])

tokens, ids, _ = workflow.step(
    tasks=[
        {'requirements': [system, q1], 'expects': ('assistant', None)}
        for _ in range(16)
    ],
    prefill=True,
    compact=False,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

example #1
The capital of Germany is Berlin.

example #2
Berlin is the capital of Germany.

example #3
The capital of Germany is Berlin.

example #4
The capital of Germany is Berlin.

example #5
The capital of Germany is Berlin.

example #6
The capital of Germany is Berlin.

example #7
The capital of Germany is Berlin.

example #8
The capital of Germany is Berlin.

example #9
The capital of Germany is Berlin.

example #10
The capital of Germany is Berlin.

example #11
The capital of Germany is Berlin.

example #12
The capital of Germany is Berlin.

example #13
The capital of Germany is Berlin.

example #14
The capital of Germany is Berlin.

example #15
The capital of Germany is Berlin.

example #16
The capital of Germany is Berlin.



## Case 7: No causality, no compaction

In [11]:
workflow.reset()

[system, q1, q2] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the questions.'''},
        'parent_ids': []
    },
    {
        'message': {'role': 'user', 'content': '''What is the capital of Germany?'''},
        'parent_ids': []
    },
    {
        'message': {'role': 'system', 'content': '''What is the smallest city in France?'''},
        'parent_ids': []
    },
])

tokens, cached = workflow.step(
    tasks=[
        {'parent_ids': [system['id'], q1['id'], q2['id']], 'expects': ('assistant', None)}
        for _ in range(16)
    ],
    prefill=True,
    compact=False,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

example #1
The capital of Germany is Berlin, not France. However, if you're looking for the smallest city in France, that's a different story.

The smallest city in France is Saint-Louis, located in the department of Moselle in the Grand Est region. It has a population of around 2,800 people and covers an area of 0.41 square kilometers (0.16 square miles).

example #2
The capital of Germany is Berlin, not France. Berlin is the largest city in Germany, not the smallest.

If you're looking for the smallest city in France, I would need more information. France has many small towns and villages, and it's difficult to determine a single smallest city. However, some of the smallest cities in France include:

* Saint-Pierre-d'Allevard, a commune in the Isère department with a population of around 1,400 people
* Saint-Pierre-d'Allevard is often cited as one of the smallest cities in France, but there are other contenders.
* La Chapelle-S

example #3
The capital of Germany is Berlin, but I thin

## Case 8: No causality, compaction Q1 -> Q2

In [12]:
workflow.reset()

[system, q1, q2] = workflow.insert([
    {
        'message': {'role': 'system', 'content': '''Answer all of the questions.'''},
        'parent_ids': []
    },
    {
        'message': {'role': 'user', 'content': '''What is the capital of Germany?'''},
        'parent_ids': []
    },
    {
        'message': {'role': 'system', 'content': '''What is the smallest city in France?'''},
        'parent_ids': []
    },
])

tokens, cached = workflow.step(
    tasks=[
        {'parent_ids': [system['id'], q1['id'], q2['id']], 'expects': ('assistant', None)}
        for _ in range(16)
    ],
    prefill=True,
    compact=True,
    max_gen_len=128,
    temperature=0.6,
    top_p=1.0,
    seed=1,
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

Not fully implemented, compact with caution!
example #1
The smallest city in France is Saint-Denis, with a population of approximately 1,000.

example #2
The capital of Germany is Berlin, not France. The capital of France is Paris.

However, if you're looking for the smallest city in France, it's difficult to give a definitive answer as there are many small towns and villages in France. But some of the smallest municipalities in France include:

* Saint-Germaine-d'Imbles, with a population of around 40 people
* Les Fauvettes, with a population of around 50 people
* Saint-Martial, with a population of around 70 people

Please note that these numbers are approximate and may have changed since the last available data.

example #3
The capital of Germany is Berlin, but it is not the smallest city in Germany. The smallest city in Germany is Elm and the smallest city in France is Celles.

example #4
The smallest city in France is Saint-Denis, which has a population of over 100,000.

example #